In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../Data"))

['custom.css', 'remove_features.pkl', 'results.zip', 'sample_submission.csv', 'test_df.pkl', 'test_identity.csv', 'test_transaction.csv', 'train_df.pkl', 'train_identity.csv', 'train_transaction.csv', '__results__.html', '__results___files', '__resultx__.html']


In [4]:
from sklearn import preprocessing
import xgboost as xgb

In [5]:
train_transaction = pd.read_csv('../Data/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('../Data/test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('../Data/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('../Data/test_identity.csv', index_col='TransactionID')

sample_submission = pd.read_csv('../Data/sample_submission.csv', index_col='TransactionID')

In [6]:
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

print(train.shape)
print(test.shape)

y_train = train['isFraud'].copy()

# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
# X_test = test.copy()
X_train = X_train.fillna(-999)
# X_test = X_test.fillna(-999)

(590540, 433)
(506691, 432)


In [7]:
from sklearn.model_selection import train_test_split
X = X_train
Y = y_train
seed = 7
test_size = 0.25
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)


In [8]:
del train, test, train_transaction, train_identity, test_transaction, test_identity

In [9]:
# Label Encoding
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))   

C:\Users\rmulsa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\rmulsa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [18]:
clf = xgb.XGBClassifier(
    n_estimators=900,
    max_depth=9,
    learning_rate=0.01,
    subsample=0.9,
    colsample_bytree=0.9,
    missing=-999,
    eval_metric = 'auc',
    random_state=2019,
    tree_method='hist',
    rate_drop =0.01
)

# clf.fit(X_train, y_train)

In [19]:
%time clf.fit(X_train, y_train)

[16:21:41] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
Wall time: 30min 19s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, eval_metric='auc', gamma=0,
       learning_rate=0.01, max_delta_step=0, max_depth=9,
       min_child_weight=1, missing=-999, n_estimators=900, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=2019,
       rate_drop=0.01, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.9, tree_method='hist')

In [20]:
# evaluate predictions
# make predictions for test data
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)
predictions = [round(value) for value in y_pred]


C:\Users\rmulsa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [12]:
sample_submission['isFraud'] = clf.predict_proba(X_test)[:,1]
sample_submission.to_csv('base_simple_xgboost.csv')

In [21]:
from sklearn.metrics import roc_auc_score
accuracy = roc_auc_score(y_test, predictions)
print("AUC: %.2f%%" % (accuracy * 100.0))

AUC: 74.76%


In [ ]:
# plt.figure()
# lw = 2
# plt.plot(fpr[2], tpr[2], color='darkorange',
#          lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
# plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic example')
# plt.legend(loc="lower right")
# plt.show()

In [56]:
# X_train['M5']

TransactionID
3397546    2
3046605    0
3570560    0
3352155    0
2988230    0
3106983    0
3482553    2
3158362    2
3040586    2
3257079    2
3328143    0
3141362    2
3300452    0
3070881    0
3305464    1
3472739    0
3569652    2
3046020    1
3532228    1
3237007    0
3211442    0
3125545    0
3534633    2
3147903    1
3308932    1
3534655    2
3363631    0
3287144    2
3410553    0
3055268    2
          ..
3526443    0
3507578    0
3399752    2
3433451    0
3493840    2
3356892    0
2991671    0
3474791    2
3402468    1
3416293    2
3447753    2
3536760    0
3195604    0
3234170    2
3179057    0
3536377    0
3532886    0
3293472    0
3492450    0
3277071    0
3270825    0
3395865    0
3053738    0
3356932    0
3392530    0
3402222    0
3065730    0
3415039    0
3560601    0
3185592    1
Name: M5, Length: 296745, dtype: int64